In [1]:
from selenium import webdriver

driver =  webdriver.Chrome("C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")
addr = "https://www.taiwan.net.tw/m1.aspx?sNo=0001123"
driver.get(addr)

In [15]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

tStart = time.time()

id_n = []
sites = []
focus = []
id_to = []
id_name = []
space = []

soup = BeautifulSoup(driver.page_source, 'html.parser')

#id
def get_id():
    for id_c in soup.find_all('a','card-link'):
        #print(id_c.get('href'))
        n = id_c.get('href')
        newid = n[23:]
        #print(newid)
        id_n.append(newid)
        space.append(" ")
    return id_n, space
        
#地點名稱       
def get_site():
    for ele in soup.select('div.card-title'):
        #print(ele.text)
        sites.append(ele.text)
    return sites
      
#點擊次數
def get_focus():
    for i in soup.select('span.target.target-like'):
        #print(i.text)
        focus.append(i.text)
    return focus

# id_to
def get_toid():
    count = 0

    for q in soup.select('div.hashtag a'):
        count += 1
        if q.text not in id_name:
            id_name.append(q.text)

    for co in range(count):
        href = soup.select('div.hashtag a')[co]['href']
        #print(href)
        newhref = href[23:]
        if newhref not in id_to:
            id_to.append(newhref)

    return id_to, id_name

In [16]:
hashtag = []
address = []
opentime = []
fro = []
to = []

def get_Info():
    for j in range(1,len(sites)+1):
        #print(len(sites)+1) # 1~42
        num = str(j)

        page = driver.find_element_by_xpath("/html/body/form/div[5]/div[3]/div[3]/ul/li[" + num + "]/div/div/a")
        #解決elenium中無法點擊Element：ElementClickInterceptedException
        driver.execute_script("arguments[0].click();", page)

        n = str(id_n[j-1])
        action = soup.find('form', id='form1').get('action')
        re_action = action.replace('./','https://www.taiwan.net.tw/')
        #取得各個景點網址
        re_a = re_action + '&id=' + n

        driver.get(re_a)
        soup2 = BeautifulSoup(driver.page_source, 'html.parser')

        #地址
        for k in soup2.select('dd a.tel-link.address'):
            address.append(k.text)
            #print(k.text)

        #開放時間
        opt = []
        if soup2.find('div','blk-zone') is not None:
            if len(soup2.find('div','blk-zone'))>2:
                for op in soup2.select('div.blk-zone p'):
                    opt.append(op.text)
                opentime.append(opt)               

            else:
                for op in soup2.select('div.blk-zone p'):
                    opentime.append(op.text)
                    #print(op.text)
        else:
            opentime.append("N/A")
            #print("NONE")

        #屬性
        tag = []

        for h in soup2.select('div.hashtag a'):
            #print(h.text)
            tag.append(h.text)
        hashtag.append(tag)
        
        #from&to
        count = 0

        for q in soup2.select('div.hashtag a'):
            count += 1

        for co in range(count):
            #id
            id_c = soup2.find('form', id='form1').get('action')
            newid = id_c[25:]
            #print(newid)
            fro.append(newid)

            href = soup2.select('div.hashtag a')[co]['href']
            #print(href)
            newhref = href[23:]
            to.append(newhref)

        driver.back()
        
    return hashtag, address, opentime, fro, to

In [17]:
import pandas as pd

i, sp = get_id()
s = get_site()
f = get_focus()
id_to, id_name= get_toid()
h, a, o, fro, to = get_Info()


#輸出為csv檔
# i+ to 可行
df = pd.DataFrame({'id': i,
                   'site_name': s,
                   'opentime': o,
                   'address': a,
                   '關注度': f,
                   '屬性': h,
                   'type': sp})
df.to_csv("site_Data.csv", index=False, encoding='big5')
# df.to_excel("site_ID.xlsx", index=False, encoding='utf_8_sig')

df2 = pd.DataFrame({'id': id_to,
                    'type': id_name})
df2.to_csv("site_attr.csv", index=False, encoding='big5')

df3 = pd.DataFrame({'from': fro,
                    'to': to})
df3.to_csv("site_Froto.csv", index=False, encoding='big5')

tEnd = time.time()
print("Run Time：",tEnd - tStart)

Run Time： 1100.1617755889893


In [18]:
# a = pd.read_csv("site_Data.csv")
# print(a)

In [19]:
# to = []
# to_name = []

# def get_to():
#     # from(id_n)
#     # to
#     count = 0

#     for q in soup.select('div.hashtag a'):
#         count += 1
#         if q.text not in to_name:
#             to_name.append(q.text)

#     for co in range(count):
#         href = soup.select('div.hashtag a')[co]['href']
#         #print(href)
#         newhref = href[23:]
#         if newhref not in to:
#             to.append(newhref)
#     return to, to_name

In [1]:
# for p in range(len(ft)):
#     print(ft[p])
#     #ft[p]
# print(len(ft))